In [13]:
from __future__ import annotations
import pandas as pd
from pathlib import Path
from typing import Dict, List
import io
import os
import numpy as np

In [14]:
# settings
log_dir = '/var/avl_logs/current/log/'

# logs to be read
# log_names = ['data_collector.log']
# leave empty to read all files
log_names = []

log_time_window = 0.1

data_collector_units = ['NAN', 'rad', 'rad', 'rad', 'm/s', 'm/s', 'm/s', 'rad', 'rad', 'm', 'rad', 'rad', 'm', 'm', 'm', 'm', 'rad/s', 'rad/s', 'rad/s', 'm/s^2', 'm/s^2', 'm/s^2', 'NAN']

# output csv filename
log_dst_csv_name = 'avl_log.csv'

In [15]:
def read_log(log_file: Path) -> Dict[str, pd.DataFrame]:
    log_strs = {}
    log_dfs = {}
    with open(log_file, "r") as f:
        data_lines = [line for line in f.readlines() if "[DAT]" in line]
        
        # TODO currently use this to prevent reading useless files
        if len(data_lines) < 20:
            return {}

        for line in data_lines:
            stream = os.path.basename(log_file)[0: -4] + '_' + line.strip().split(' ')[3][1: -1]
            if stream not in log_strs:
                log_strs[stream] = []
            data = line.strip().split(' ')
            time = data[0].replace("[", "").replace("]", "")
            log_strs[stream].append([time] + data[4:])

        for stream, data in log_strs.items():
            names = data.pop(0)[1:]
            units = data.pop(0)[1:]
            col_names = ["time"] + [f"[{stream}]{name}_{unit}" for name, unit in zip(names, units)]
            log_dfs[stream] = pd.DataFrame(data, columns=col_names)

    return log_dfs

def read_data_collector_log(log_file: Path) -> Dict[str, pd.DataFrame]:
    stream = 'data_collector'
    log_df = pd.read_csv(log_file, delim_whitespace=True)
    log_df = log_df.drop(index=0, axis=0).reset_index(drop=True)
    columns = list(log_df.columns)
    if len(columns) - 1 != len(data_collector_units):
        print('data_collector_units not match error')
        return
    
    columns[0] = 'time'
    for i in range(0, len(data_collector_units)):
            columns[i + 1] = f'[{stream}]{columns[i + 1]}_{data_collector_units[i]}'
    log_df.columns = columns
    
    log_dfs = {}
    log_dfs[stream] = log_df
    return log_dfs

# Read each log file into a DataFrame
# log_data: Dict[str, pd.DataFrame] = {}
# for log_file in log_files:
#     ...
# Read required the log files and put their contents into the log_lines dict
# Grab required files in log folder that end in .log
if len(log_names) == 0:
    log_files = [log_file for log_file in Path(log_dir).iterdir() if log_file.is_file() and log_file.suffix == '.log']
else:
    log_files = [Path(log_dir + log_name) for log_name in log_names if Path(log_dir + log_name).is_file and Path(log_dir + log_name).suffix == '.log']

log_raw_data = {}
for log_file in log_files:
    if os.path.basename(log_file) == 'data_collector.log':
        log_dfs = read_data_collector_log(log_file)
    elif os.path.basename(log_file) == 'inertial_nav_node.log': # TODO currently don't know why this can't read this file
        log_dfs = {}
    elif os.path.basename(log_file) == 'ethernet_channel_node.log': # TODO this file's format is different and I think this file is useless
        log_dfs = {}
    else:
        log_dfs = read_log(log_file)
    for stream, data in log_dfs.items():
        log_raw_data[stream] = log_dfs[stream]


In [16]:
log_df = None
log_df_created = False
log_min_time = max([float(raw_df.at[0, 'time']) for _, raw_df in log_raw_data.items()])
log_max_time = min([float(raw_df.at[len(raw_df) - 1, 'time']) for _, raw_df in log_raw_data.items()])

for stream, raw_df in log_raw_data.items():
    log_time = log_min_time
    parsed_df = pd.DataFrame(columns=raw_df.columns)
    cur_row = 0
    
    while log_time < log_max_time:
        while float(raw_df.at[cur_row, 'time']) >= log_time and log_time < log_max_time:
            log_time += log_time_window
            parsed_df.loc[len(parsed_df)] = [str(log_time)] + list(raw_df.loc[cur_row][1:])
        else:
            cur_row += 1
    
    if log_df_created:
        log_df = pd.concat([log_df, parsed_df.drop('time', axis=1)], axis=1)
        print(f'finished processing {stream}')
    else:
        log_df = parsed_df
        log_df_created = True

display(log_df)
log_df.to_csv(log_dst_csv_name, index=False)

finished processing depth_sim_node_depth
finished processing dynamics_manager_node_nu
finished processing dynamics_manager_node_nu_dot
finished processing dynamics_manager_node_eta
finished processing dynamics_manager_node_eta_dot
finished processing dynamics_manager_node_p_b
finished processing dynamics_manager_node_w_ib_b
finished processing dynamics_manager_node_f_ib_b
finished processing actuators_node_actuators
finished processing ahrs_sim_node_theta_n_b
finished processing ahrs_sim_node_f_ib_b
finished processing ahrs_sim_node_w_ib_b
finished processing ahrs_sim_node_m_b
finished processing imu_sim_node_f_ib_b
finished processing imu_sim_node_w_ib_b
finished processing dvl_sim_node_v_eb_b
finished processing pid_attitude_control_node_roll_pid
finished processing pid_attitude_control_node_pitch_pid
finished processing pid_attitude_control_node_yaw_pid
finished processing pid_attitude_control_node_roll_info
finished processing pid_attitude_control_node_pitch_info
finished processin

,time,[lbl_range_sim_node_beacon_0]lat_deg,[lbl_range_sim_node_beacon_0]lon_deg,[lbl_range_sim_node_beacon_0]alt_m,[lbl_range_sim_node_beacon_0]range_m,[depth_sim_node_depth]depth_m,[dynamics_manager_node_nu]u_m/s,[dynamics_manager_node_nu]v_m/s,[dynamics_manager_node_nu]w_m/s,[dynamics_manager_node_nu]p_rad/s,...,[rpm_control_node_info]error_rpm,[rpm_control_node_info]p_output_%,[rpm_control_node_info]i_output_%,[rpm_control_node_info]d_output_%,[rpm_control_node_info]i_error_sum_%,[rpm_control_node_info]output_%,[rpm_control_node_info]unclamped_output_%,[rpm_control_node_info]output_saturated_bool,[rpm_control_node_info]i_saturated_bool,[rpm_sim_node_rpm]RPM_RPM
0,1679349139.18148,0.646806,-1.406988,-10.000000,143.220,-0.026646675,0.000001,0.000000,-0.000000,0.000000,...,2000.000000,0.000000,3.200000,-0.000000,400.000000,3.200000,3.200000,0,0,128.000000000
1,1679349139.2814798,0.646806,-1.406988,-10.000000,143.220,-0.038823381,0.000162,0.000000,-0.000000,0.000000,...,1872.000000,0.000000,12.492800,-0.000000,1561.600000,12.492800,12.492800,0,0,279.808000000
2,1679349139.3814797,0.646806,-1.406988,-10.000000,143.220,0.018838054,0.001031,0.000000,-0.000000,0.000000,...,1720.192000,0.000000,22.247322,-0.000000,2780.915200,22.247322,22.247322,0,0,389.900288000
3,1679349139.4814796,0.646806,-1.406988,-10.000000,143.220,-0.002919688,0.003360,0.000000,-0.000000,0.000000,...,1452.007186,0.000000,30.884458,-0.000000,3860.557241,30.884458,30.884458,0,0,664.153388483
4,1679349139.5814795,0.646806,-1.406988,-10.000000,143.220,0.008279747,0.007856,0.000000,-0.000001,0.000000,...,1335.846612,0.000000,39.712662,-0.000000,4964.082702,39.712662,39.712662,0,0,772.879686603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,1679349153.781466,0.646806,-1.406988,-10.000000,116.849,0.000000000,1.984952,0.010351,0.008381,0.003298,...,0.000175,0.000000,99.999992,-0.000000,12499.998998,99.999992,99.999992,0,0,1999.999842526
147,1679349153.881466,0.646806,-1.406988,-10.000000,116.849,0.000000000,1.985710,0.010184,0.008447,0.003519,...,0.000157,0.000000,99.999993,-0.000000,12499.999079,99.999993,99.999993,0,0,1999.999855124
148,1679349153.9814658,0.646806,-1.406988,-10.000000,116.849,0.000000000,1.986259,0.010167,0.008620,0.003654,...,0.000145,0.000000,99.999993,-0.000000,12499.999168,99.999993,99.999993,0,0,1999.999866915
149,1679349154.0814657,0.646806,-1.406988,-10.000000,116.849,0.000000000,1.986785,0.010463,0.008760,0.003739,...,0.000122,0.000000,99.999994,-0.000000,12499.999273,99.999994,99.999994,0,0,1999.999887528
